### Running DeepSeek-Janus-Pro-1B on Google Colab GPU

In [1]:
# clonign the deepseek-janus-pro-1b repo
!git clone https://github.com/deepseek-ai/Janus.git

Cloning into 'Janus'...
remote: Enumerating objects: 118, done.
remote: Counting objects: 100% (72/72), done.
remote: Compressing objects: 100% (36/36), done.
remote: Total 118 (delta 50), reused 36 (delta 36), pack-reused 46 (from 1)
Receiving objects: 100% (118/118), 7.18 MiB | 12.47 MiB/s, done.
Resolving deltas: 100% (56/56), done.


In [2]:
## Installing dependencues
%cd Janus
!pip3 install -e .
!pip3 install flash-attn

/content/Janus
Obtaining file:///content/Janus
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 71.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 55.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 46.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━

In [7]:
# importing dependencies
import torch
import transformers
from transformers import AutoModelForCausalLM
from janus.models import MultiModalityCausalLM
from janus.models import VLChatProcessor
from janus.utils.io import load_pil_images

##### Loading the Model and moving it to CUDA(GPU)

In [8]:
# defining model
modelPath = 'deepseek-ai/Janus-Pro-1B'
# loading processor and tokenizer
vlChatProcessor = VLChatProcessor.from_pretrained(modelPath)
tokenizer = vlChatProcessor.tokenizer

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/346 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/285 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/4.72M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/344 [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama_fast.LlamaTokenizerFast'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565 - if you loaded a llama tokenizer from a GGUF file you can ignore this message.


processor_config.json:   0%|          | 0.00/210 [00:00<?, ?B/s]

Some kwargs in processor config are unused and will not have any effect: add_special_token, image_tag, ignore_id, mask_prompt, num_image_tokens, sft_format. 


In [9]:
# loading model with remote code enabled
vlGPT = AutoModelForCausalLM.from_pretrained(modelPath, trust_remote_code=True)
# Moving model to GPU memory
vlGPT = vlGPT.to(torch.bfloat16).cuda().eval()

config.json:   0%|          | 0.00/1.46k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/4.18G [00:00<?, ?B/s]

### Passing images to model for processing

  - Place your images inside Janus/images folder.

In [16]:
!ls images | grep group

group_photo.jpg


In [13]:
imagePath = "images/group_photo.jpg"
query = "Explain this image in detail"

conversation = [{
    'role': '<|User|>', 'content': f'<image_placeholder>\n{query}',
    'images': [imagePath]
    },
    {'role': '<|Assistant|>', 'content': ''}
]

##### Processing the Image

In [14]:
# laoding image
pilImage = load_pil_images(conversation)

# crafting input for model
creaftInput = vlChatProcessor(conversations=conversation, images = pilImage, force_batchify=True,).to(vlGPT.device)
inputsEmbeds = vlGPT.prepare_inputs_embeds(**creaftInput)

In [18]:
# Generating respoinse
outputs = vlGPT.language_model.generate(
    use_cache = True,
    do_sample = False,
    max_new_tokens = 512,
    inputs_embeds = inputsEmbeds,
    pad_token_id = tokenizer.eos_token_id,
    bos_token_id = tokenizer.bos_token_id,
    eos_token_id =  tokenizer.eos_token_id,
    attention_mask = creaftInput.attention_mask,
)

In [19]:
## Decode the response
answer = tokenizer.decode(outputs[0].cpu().tolist(), skip_special_tokens=True)
print(f"{creaftInput['sft_format'][0]}", answer)

You are a helpful language and vision assistant. You are able to understand the visual content that the user provides, and assist the user with a variety of tasks using natural language.

<|User|>: <image_placeholder>
Explain this image in detail

<|Assistant|>: The image shows four men standing on a grassy area with a scenic mountain backdrop. They appear to be on a hiking or outdoor excursion.

1. **First Person (Left)**:
   - **Clothing**: Wearing a light grey long-sleeve sweater, blue jeans, and white sneakers.
   - **Accessories**: He is holding a bouquet of yellow flowers in his right hand.

2. **Second Person (Second from Left)**:
   - **Clothing**: Wearing a brown jacket over a black shirt, blue jeans, and dark sneakers.
   - **Accessories**: He has sunglasses on his head.

3. **Third Person (Second from Right)**:
   - **Clothing**: Wearing a black Gucci sweatshirt with a logo on the chest, blue jeans, and dark sneakers.
   - **Accessories**: He is also wearing sunglasses.

4. 